## Прогнозирование временных рядов на карте
Ваш проект почти закончен! На этой неделе вам предстоит оформить результат, используя интерактивные графики.

### Инструкции

Определитесь, в каком виде вы хотите представить ваш проект. Мы предлагаем вам сделать интерактивное демо, содержащее:

* карты с визуализацией реального и прогнозируемого спроса на такси в выбираемый пользователем момент времени
* временной ряд фактического и прогнозируемого спроса на такси в выбираемой области.

Немного материалов о том, как в ноутбуках можно делать интерактивные графики:
* https://blog.dominodatalab.com/interactive-dashboards-in-jupyter/
* http://nbviewer.jupyter.org/github/quantopian/ipython/blob/master/examples/Interactive%20Widgets/Index.ipynb
* https://github.com/ioam/holoviews

In [ ]:
import pandas as pd
import numpy as np
import datetime
from StringIO import StringIO

%matplotlib inline

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()
from plotly.offline import iplot
from plotly import tools

from IPython.display import display, clear_output
from traitlets import link
from ipywidgets import widgets

import folium
import json

In [ ]:
data = pd.read_csv('taxi_data.csv.gz')
data['dt'] = pd.to_datetime(data.dt)

regions = list(np.sort(np.unique(data.region.values), axis=None))
steps = range(1,7)

### Временные ряды для выбранной зоны

In [ ]:
def plot_timeline(data, region, step):
    """
    Makes prediction and actual timelines
    """
    data_region=data[data.region==region].sort_values('dt')
    x=data_region.dt+datetime.timedelta(hours=step)
    trace_real = go.Scatter(x=x, y=data_region['y_'+str(step)], mode='lines', name='real', line=dict(width=1.5))
    trace_pred = go.Scatter(x=x, y=data_region['yp_'+str(step)], mode='lines', name='prediction', 
                            line=dict(width=1.5)
                           )
    layout = go.Layout(
        title='Real and prediction timeseries with step=%s and region=%s' % (step, region),
        xaxis=dict(
            rangeslider=dict(),
            rangeselector=dict(
                buttons=list([
                        dict(count=1, label='d', step='day', stepmode='todate'),
                        dict(count=7, label='w', step='day', stepmode='backwards'),
                        dict(step='all')
                ])
            )
        )
    )
    fig=go.Figure(data=[trace_real, trace_pred], layout=layout)
    iplot(fig)

In [ ]:
cmb_regions=widgets.Dropdown(
    options=regions,
    value=regions[0],
    description='Region:',
)

sld_steps=widgets.IntSlider(value=1, min=1, max=6, step=1, description='Step')

pnl=widgets.HBox(children=[cmb_regions, sld_steps])
pnl.width = '100%'
pnl.pack = 'center'

display(pnl)

def change_values(sender):
    """
    handles the value changing
    """
    clear_output()
    plot_timeline(data=data, region=cmb_regions.value, step=sld_steps.value)
    
cmb_regions.observe(change_values, 'value')
sld_steps.observe(change_values, 'value')
plot_timeline(data=data, region=regions[0], step=steps[0])

### Карта прогнозов в выбранный час (folium)

In [ ]:
num_steps = 50
west=-74.25559
east=-73.70001
north=40.91553
south=40.49612
url_base = 'http://server.arcgisonline.com/ArcGIS/rest/services/'
service = 'NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}'
tileset = url_base + service

regionsDF = pd.read_csv('./regions.csv', delimiter=';')

In [ ]:
geo_data=data.merge(regionsDF, on='region')
(min_hour, max_hour)=(geo_data.hour_no.min(), geo_data.hour_no.max())

In [ ]:
def plot_hour(data, hour_no=min_hour, step_no=1, is_pred=True):
    """
    Makes folium map with colored regions with selected hour
    """
    grid = {"type": "FeatureCollection", "features":[]}
    filtered = data[data.hour_no==hour_no]
    for (i, row) in filtered.iterrows():
        (w, e, s, n) = (row['west'], row['east'], row['south'], row['north'])
        coords = [[w, n],[e, n],[e, s],[w, s],[w, n]]
        grid['features'].append({"type": "Feature",
                                 "id": row['region'],
                                 "geometry": {
                                     "type": "Polygon",
                                     "coordinates": [coords]},
                                 "properties": {"name": str(row['region'])}
                                }
                               )
    io = StringIO()
    json.dump(grid, io)
    
    flm_map = folium.Map(location=[(north+south)/2, (east+west)/2], zoom_start=11,
                     control_scale = True, tiles=tileset, attr='USGS style')
    
    data_col_name = 'yp_%s' % step_no if is_pred else 'y_%s' % step_no
    fill_color = 'BuPu' if is_pred else 'YlOrRd'
    flm_map.choropleth(geo_str=io.getvalue(), data=filtered,
              columns=['region', data_col_name],
              threshold_scale=[50, 100, 200, 400, 600, 800],
              key_on='feature.id',
              fill_color=fill_color, fill_opacity=0.6,
              line_opacity=0.1, line_weight=0.2)
    
    return flm_map

In [ ]:
sld_hour=widgets.IntSlider(
    value=min_hour, min=min_hour, max=max_hour, description='Hour No:'
)

sld_steps_1=widgets.IntSlider(value=1, min=1, max=6, step=1, description='Step:')
chk_pred=widgets.Checkbox(value=True, description='Prediction')

pnl_1=widgets.HBox(children=[sld_hour, sld_steps_1, chk_pred])
pnl_1.width = '100%'
pnl_1.pack = 'center'

display(pnl_1)

def change_map(sender):
    """
    handles the value changing
    """
    clear_output()
    flm_map=plot_hour(data=geo_data, hour_no=sld_hour.value, step_no=sld_steps_1.value, is_pred=chk_pred.value)
    display(flm_map)
    
sld_hour.observe(change_map, 'value')
sld_steps_1.observe(change_map, 'value')
chk_pred.observe(change_map, 'value')
flm_map=plot_hour(data=geo_data, hour_no=sld_hour.value, step_no=sld_steps_1.value, is_pred=chk_pred.value)
display(flm_map)